In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Feb 10 08:57:52 2022

@author: natakranp and suriya :)
"""

import pandas as pd

import requests
from requests.auth import HTTPBasicAuth
import json

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

# ####################################################

df_tags = pd.read_csv('MAC-PredictiveMA-RUL-Tags-Desciption.csv',index_col=0)

# ####################################################

#วันเริ่มดึง
start_date = "2021-12-01"
ended_date = "2022-05-30"
ar_start_date = start_date.split("-")
date_begin = date( int(ar_start_date[0]), int(ar_start_date[1]), int(ar_start_date[2]))

start = datetime.strptime(start_date, "%Y-%m-%d")
end = datetime.strptime(ended_date, "%Y-%m-%d")
date_generated = pd.date_range(start, end)
data_range  = date_generated.strftime("%Y-%m-%d").to_list()

#จำนวนวันที่ต้องการดึง
total_days = len(data_range)

PATH_OUTPUT_FOLDER = '../src/RawData_MKS/'

for index, row in df_tags.iterrows():
    df_main = pd.DataFrame()

    TagNameClean = str( row["Descriptor"].replace(' ','-').replace('%',"_%_").replace('_%_',"_Percent_").replace('Mill_B',"").replace('.',"").replace('/',"").replace("'","") )
    Id = row['Id']
    Name = row['Name'].replace('/','-')
    Mill = 'MKS'
    print(TagNameClean, Name)

    for i in range(total_days):
        date_start = date_begin + relativedelta(days=i)
        date_ended = date_start + relativedelta(days=1) - relativedelta(seconds=1)
        str_date_start = date_start.strftime("%Y-%m-%dT%H:%M:%SZ")
        str_date_ended = date_ended.strftime("%Y-%m-%dT%H:%M:%SZ")

        get_type = 'recorded' # recorded , interpolated , plot , summary , value , end

        WebID = row['WebId'] #'F1DP7qjp812TukOCueRElcymMwUXgAAATVBHREMtUElNU0RCXDMwMzEtRENTLU1MQi1USS0wODEx'
        url_hda = f'https://pims.mitrphol.com/piwebapi/streams/{WebID}/{get_type}?startTime={str_date_start}&endtime={str_date_ended}'

        res = requests.get(url_hda)
        parsed_json = json.loads( res.text )
        df = pd.DataFrame.from_dict(parsed_json['Items'], orient='columns')
        df = df.rename(columns = {'Value': f'{Name}_{TagNameClean}'}, inplace = False)
        # print(len(df))
        df_main = df_main.append(df, ignore_index = True)

        if len(df) == 1000:
            while 1:
                # print(df.tail(1)['Timestamp'].iloc[0])
                str_date_start = df.tail(1)['Timestamp'].iloc[0]
                str_date_ended = date_ended.strftime("%Y-%m-%dT%H:%M:%SZ")

                get_type = 'recorded'

                WebID = row['WebId']
                url_hda = f'https://pims.mitrphol.com/piwebapi/streams/{WebID}/{get_type}?startTime={str_date_start}&endtime={str_date_ended}'

                res = requests.get(url_hda)
                parsed_json = json.loads( res.text )
                df = pd.DataFrame.from_dict(parsed_json['Items'], orient='columns')
                df = df.rename(columns = {'Value': f'{Name}_{TagNameClean}'}, inplace = False)
                df_main = df_main.append(df.iloc[1:], ignore_index = True)
                if len(df) < 1000:
                    break

    print(TagNameClean, len(df_main))

    if len(df_main) !=0:
        df_main['Datetime'] = pd.to_datetime(df_main['Timestamp'], format="%Y-%m-%d %H:%M:%S")

        first_column = df_main.pop('Datetime')
        df_main.insert(0, 'Datetime', first_column)

        df_main.to_csv(f'{PATH_OUTPUT_FOLDER}/{Mill}_{Name}_{date_begin.strftime("%Y-%m-%d")}_{date_ended.strftime("%Y-%m-%d")}_{TagNameClean}.csv')
        df_main = df_main.query('Good != False')
        plt.figure(figsize=(20, 8))
        plt.plot(df_main['Datetime'], df_main[f'{Name}_{TagNameClean}'])

        plt.xticks(rotation = 45)
        plt.xlabel("Datetime")
        plt.ylabel(f'{Name}_{TagNameClean}')
        plt.title(f'{Name}_{TagNameClean} (Sampling:{len(df_main)})')

        plt.savefig(f'{PATH_OUTPUT_FOLDER}/{Mill}_{Name}_{date_begin.strftime("%Y-%m-%d")}_{date_ended.strftime("%Y-%m-%d")}_{TagNameClean}.png')




Shredder-Flow-Lube-Oil-NDE Q0Q1-DCS-ML-105FT001


JSONDecodeError: Expecting value: line 1 column 1 (char 0)